# MSR 2018 Challenge Notebook
This notebook contains the work done for the analysis for Tyson Bulmer's MSR 2018 Challgne paper using the supplied dataset.

In [1]:
import psycopg2
import psycopg2.extras
import pandas as pd
import numpy as np

# To deal with right skewness we can take the log of the values
import math

import matplotlib.pyplot as plt

from tqdm import tqdm

# Configure display of dataframe visualizations
pd.options.display.mpl_style = 'default'

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2862: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


## Connect to database and get event data

In [3]:
conn = psycopg2.connect("dbname='msr2018' user='Tyson' host='localhost' password=''")

In [4]:
cur = conn.cursor()

In [5]:
cur.execute("""select event_type, idesessionuuid, triggeredat, commandid, cancelled, terminatedstate, action, typeofnavigation, wasaborted from events""")

In [6]:
colnames = [desc[0] for desc in cur.description]

In [7]:
results = cur.fetchall()

In [8]:
results = pd.DataFrame([list(row) for row in results], columns=colnames)

In [9]:
# Clean up events column strings
results['event_type'] = results['event_type'].apply(lambda x:'.'.join(x.split(',')[0].split('.')[4:]))

In [10]:
# Filter results to only applicable events
events_to_use = [
    'CommandEvent', 'NavigationEvent', 'VisualStudio.WindowEvent',
    'CompletionEvents.CompletionEvent',
       'VisualStudio.EditEvent', 'VisualStudio.DocumentEvent',
       'VisualStudio.DebuggerEvent',
       'VisualStudio.SolutionEvent', 'VisualStudio.IDEStateEvent', 'VisualStudio.BuildEvent',
       'TestRunEvents.TestRunEvent',
       'VersionControlEvents.VersionControlEvent', 'VisualStudio.FindEvent'
]
results = results[results['event_type'].isin(events_to_use)].fillna('')

In [11]:
#Keep only the sessions which have multiple entries
results = results[results.duplicated(subset=['idesessionuuid'], keep=False)]

In [12]:
# Clean up commandid column strings
results['commandid'] = results['commandid'].apply(lambda x:'.'.join(x.split(':')[-1].split('.')[-2:]))

In [13]:
results = results.sort_values('triggeredat')
results

,event_type,idesessionuuid,triggeredat,commandid,cancelled,terminatedstate,action,typeofnavigation,wasaborted
3720321,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.289266,,,,2,,
3784862,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.290267,,,,2,,
3780848,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.290267,,,,2,,
3776456,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.619717,,,,2,,
3744516,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.620718,,,,2,,
3702408,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.620718,,,,2,,
3788390,CommandEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.428542,Edit.BreakLine,,,,,
3738583,CommandEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.430543,TextControl.Enter,,,,,
3791738,VisualStudio.EditEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.432544,,,,,,
3779039,CommandEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.587183,Edit.BreakLine,,,,,


In [14]:
d = {'CommandEvent':'commandid',
     'CompletionEvents.CompletionEvent':'terminatedstate',
     'VisualStudio.DocumentEvent': 'action',
     'VisualStudio.FindEvent':'cancelled',
     'VisualStudio.SolutionEvent':'action',
     'VisualStudio.WindowEvent': 'action',
     'NavigationEvent': 'typeofnavigation',
     'TestRunEvents.TestRunEvent': 'wasaborted'
    }

d_keys = d.keys()
    
results['event_type_complete'] = results.apply(lambda x: x['event_type']+'-'+str(x[d[x['event_type']]]) if x['event_type'] in d_keys else x['event_type'], axis=1)

In [15]:
results

,event_type,idesessionuuid,triggeredat,commandid,cancelled,terminatedstate,action,typeofnavigation,wasaborted,event_type_complete
3720321,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.289266,,,,2,,,VisualStudio.WindowEvent-2
3784862,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.290267,,,,2,,,VisualStudio.WindowEvent-2
3780848,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.290267,,,,2,,,VisualStudio.WindowEvent-2
3776456,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.619717,,,,2,,,VisualStudio.WindowEvent-2
3744516,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.620718,,,,2,,,VisualStudio.WindowEvent-2
3702408,VisualStudio.WindowEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.620718,,,,2,,,VisualStudio.WindowEvent-2
3788390,CommandEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.428542,Edit.BreakLine,,,,,,CommandEvent-Edit.BreakLine
3738583,CommandEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.430543,TextControl.Enter,,,,,,CommandEvent-TextControl.Enter
3791738,VisualStudio.EditEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.432544,,,,,,,VisualStudio.EditEvent
3779039,CommandEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.587183,Edit.BreakLine,,,,,,CommandEvent-Edit.BreakLine


In [16]:
results = results[['event_type_complete', 'idesessionuuid', 'triggeredat']]

In [17]:
results

,event_type_complete,idesessionuuid,triggeredat
3720321,VisualStudio.WindowEvent-2,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.289266
3784862,VisualStudio.WindowEvent-2,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.290267
3780848,VisualStudio.WindowEvent-2,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:48:47.290267
3776456,VisualStudio.WindowEvent-2,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.619717
3744516,VisualStudio.WindowEvent-2,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.620718
3702408,VisualStudio.WindowEvent-2,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:53:44.620718
3788390,CommandEvent-Edit.BreakLine,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.428542
3738583,CommandEvent-TextControl.Enter,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.430543
3791738,VisualStudio.EditEvent,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.432544
3779039,CommandEvent-Edit.BreakLine,fef6f3d6-3a5f-4f7c-8ddf-b52346a831c5,2016-04-02 21:57:34.587183


In [18]:
# Save to a file so we can cache load it
results.to_csv('results.csv', index=False)

In [19]:
groups = results.groupby('idesessionuuid')

In [20]:
for sessionid, group in groups:
    group.to_csv('./sessions/'+sessionid + '.csv')

In [2]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('./sessions/'):
    f = filenames
    break
# filenames

In [ ]:
from os import walk
import pandas as pd

f = []
for (dirpath, dirnames, filenames) in walk('./sessions/'):
    f = filenames
    break
data = []
for f in filenames:
    group = pd.read_csv('./sessions/' +f)
#     for index, group in pd.read_csv(f).groupby('idesessionuuid', as_index=False):
    events = group['event_type_complete'].tolist()
    indices = [i for i, x in enumerate(events) if x.split('-')[0] == "CommandEvent"]

    for ind in indices:
        x = events[:ind][:-10] # Max length of event chains to 10
        y = events[ind]
        if len(x) > 0:
            data.append([' '.join(x), y])
del group
del events
del indices
    
data = pd.DataFrame(data, columns=['events', 'command'])
data.to_csv('data.csv', index=False)

In [ ]:
data